### Execução do algoritmo Apriori - Biblioteca mlxtend
http://rasbt.github.io/mlxtend/

# Bibliotecas

In [431]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from decimal import Decimal

# Carregamento dos Dados - .csv

In [432]:
endereco = '../BaseDadosSelecionados/'
# endereco2 = 'Agrupado_Ocorrencia'
nomeArq = 'ES_datatran2019_Apriori'

In [433]:
data = pd.read_csv(endereco + nomeArq + '.csv', encoding='ISO-8859-1', sep=';', header=None)

In [434]:
data.shape

(2842, 21)

In [435]:
data.head(5)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,dia_semana=terça-feira,br=101.0,km=121.3,municipio=SOORETAMA,causa_acidente=Condutor Dormindo,tipo_acidente=Colisão lateral,classificacao_acidente=Com Vítimas Feridas,fase_dia=Pleno dia,sentido_via=Crescente,condicao_metereologica=Céu Claro,...,tracado_via=Reta,uso_solo=0,pessoas=2,mortos=0,feridos_leves=0,feridos_graves=1,ilesos=1,ignorados=0,feridos=1,veiculos=2
1,dia_semana=terça-feira,br=262.0,km=95,municipio=DOMINGOS MARTINS,causa_acidente=Não guardar distância de segurança,tipo_acidente=Engavetamento,classificacao_acidente=Sem Vítimas,fase_dia=Pleno dia,sentido_via=Decrescente,condicao_metereologica=Céu Claro,...,tracado_via=Reta,uso_solo=1,pessoas=4,mortos=0,feridos_leves=0,feridos_graves=0,ilesos=4,ignorados=0,feridos=0,veiculos=4
2,dia_semana=terça-feira,br=101.0,km=77.1,municipio=SAO MATEUS,causa_acidente=Falta de Atenção à Condução,tipo_acidente=Colisão traseira,classificacao_acidente=Com Vítimas Feridas,fase_dia=Pleno dia,sentido_via=Decrescente,condicao_metereologica=Céu Claro,...,tracado_via=Desvio Temporário,uso_solo=0,pessoas=3,mortos=0,feridos_leves=1,feridos_graves=1,ilesos=1,ignorados=0,feridos=2,veiculos=2
3,dia_semana=terça-feira,br=101.0,km=299.2,municipio=VIANA,causa_acidente=Desobediência às normas de trân...,tipo_acidente=Colisão transversal,classificacao_acidente=Sem Vítimas,fase_dia=Anoitecer,sentido_via=Decrescente,condicao_metereologica=Céu Claro,...,tracado_via=Não Informado,uso_solo=1,pessoas=3,mortos=0,feridos_leves=0,feridos_graves=0,ilesos=3,ignorados=0,feridos=0,veiculos=2
4,dia_semana=terça-feira,br=262.0,km=168,municipio=IBATIBA,causa_acidente=Falta de Atenção à Condução,tipo_acidente=Queda de ocupante de veículo,classificacao_acidente=Com Vítimas Feridas,fase_dia=Anoitecer,sentido_via=Crescente,condicao_metereologica=Céu Claro,...,tracado_via=Reta,uso_solo=0,pessoas=1,mortos=0,feridos_leves=0,feridos_graves=1,ilesos=0,ignorados=0,feridos=1,veiculos=1


# Carregamento dos dados em uma matriz (Importante)

In [436]:
%%time
# data.shape[0] LINHAS e data.shape[1] COLUNAS

dataset = []
for i in range(0, data.shape[0]):
    dataset.append([str(data.values[i,j]) for j in range(0, data.shape[1])])

Wall time: 934 ms


In [437]:
# dataset

# Tratamento para a execução do algoritmo Apriori

http://rasbt.github.io/mlxtend/user_guide/preprocessing/TransactionEncoder/

In [438]:
%%time
te = TransactionEncoder()

Wall time: 0 ns


## 1º Transformação - Representação para uma matriz de boleanos

In [439]:
%%time
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

Wall time: 66 ms


In [440]:
# df

## 2ª Transformação - Representação esparsa
Exemplo 3 - Trabalhando com representações esparsas - http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/ 

In [441]:
# %%time
# oht_ary = te.fit(dataset).transform(dataset, sparse=True)
# sparse_df = pd.DataFrame.sparse.from_spmatrix(oht_ary, columns=te.columns_)

In [442]:
# sparse_df

# Execução do algoritmo apriori

In [443]:
numMinSupport = 0.1

In [444]:
%%time
# sparse_df
frequent_itemsets = apriori(df, min_support=numMinSupport, use_colnames=True)

Wall time: 8.24 s


In [445]:
# frequent_itemsets

In [446]:
# metric = lift

In [447]:
%%time
association_rules_saida = association_rules(frequent_itemsets, metric="confidence", min_threshold=1.0)
# association_rules_saida = association_rules(frequent_itemsets)

Wall time: 6.17 s


In [448]:
saidaRecords = pd.DataFrame(association_rules_saida)

## Método para contabilizaro número de consequentes e antecedentes em cada regra

In [449]:
# Cria duas novas colunas contabilizando os antecedentes e consequents
saidaRecords["antecedent_len"] = saidaRecords["antecedents"].apply(lambda x: len(x))
saidaRecords["consequent_len"] = saidaRecords["consequents"].apply(lambda x: len(x))

## Filtro

In [450]:
saidaRecords = saidaRecords[ (saidaRecords['antecedent_len'] >= 3 ) & (saidaRecords['consequent_len'] >= 3) ] # & (saidaRecords['support'] >= 0.5) 

In [451]:
# saidaRecords = saidaRecords[(saidaRecords['support'] >= 0.6)]

In [452]:
saidaRecords.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
4073,"(feridos=1, br=101.0, pessoas=1)","(ignorados=0, ilesos=0, classificacao_acidente...",0.130542,0.286066,0.130542,1.0,3.495695,0.093198,inf,3,3
4074,"(pessoas=1, br=101.0, classificacao_acidente=C...","(ignorados=0, feridos=1, ilesos=0)",0.130542,0.183673,0.130542,1.0,5.444444,0.106565,inf,3,3
4101,"(feridos=1, br=101.0, pessoas=1)","(ignorados=0, mortos=0, classificacao_acidente...",0.130542,0.732231,0.130542,1.0,1.365690,0.034955,inf,3,3
4102,"(pessoas=1, br=101.0, classificacao_acidente=C...","(ignorados=0, feridos=1, mortos=0)",0.130542,0.525686,0.130542,1.0,1.902276,0.061918,inf,3,3
4132,"(feridos=1, br=101.0, pessoas=1)","(ignorados=0, veiculos=1, classificacao_aciden...",0.130542,0.294511,0.130542,1.0,3.395460,0.092096,inf,3,3
4133,"(pessoas=1, br=101.0, classificacao_acidente=C...","(feridos=1, veiculos=1, ignorados=0)",0.130542,0.224842,0.130542,1.0,4.447574,0.101191,inf,3,3
4145,"(br=101.0, feridos=1, pessoas=1)","(mortos=0, ilesos=0, classificacao_acidente=Co...",0.130542,0.346587,0.130542,1.0,2.885279,0.085298,inf,3,3
4146,"(pessoas=1, br=101.0, classificacao_acidente=C...","(feridos=1, mortos=0, ilesos=0)",0.130542,0.225897,0.130542,1.0,4.426791,0.101053,inf,3,3
4160,"(br=101.0, feridos=1, pessoas=1)","(veiculos=1, ilesos=0, classificacao_acidente=...",0.130542,0.260732,0.130542,1.0,3.835358,0.096505,inf,3,3
4161,"(pessoas=1, br=101.0, classificacao_acidente=C...","(veiculos=1, feridos=1, ilesos=0)",0.130542,0.194581,0.130542,1.0,5.139241,0.105141,inf,3,3


In [453]:
saidaRecords.max()

antecedents           (pessoas=2, uso_solo=1, feridos_leves=1, veicu...
consequents           (feridos=1, mortos=0, classificacao_acidente=C...
antecedent support                                             0.267065
consequent support                                             0.732231
support                                                        0.267065
confidence                                                            1
lift                                                            8.00563
leverage                                                       0.148373
conviction                                                          inf
antecedent_len                                                        7
consequent_len                                                        6
dtype: object

## Salvando os resultados do algoritmo em um .csv

In [454]:
saidaRecords.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
4073,"(feridos=1, br=101.0, pessoas=1)","(ignorados=0, ilesos=0, classificacao_acidente...",0.130542,0.286066,0.130542,1.0,3.495695,0.093198,inf,3,3
4074,"(pessoas=1, br=101.0, classificacao_acidente=C...","(ignorados=0, feridos=1, ilesos=0)",0.130542,0.183673,0.130542,1.0,5.444444,0.106565,inf,3,3
4101,"(feridos=1, br=101.0, pessoas=1)","(ignorados=0, mortos=0, classificacao_acidente...",0.130542,0.732231,0.130542,1.0,1.365690,0.034955,inf,3,3
4102,"(pessoas=1, br=101.0, classificacao_acidente=C...","(ignorados=0, feridos=1, mortos=0)",0.130542,0.525686,0.130542,1.0,1.902276,0.061918,inf,3,3
4132,"(feridos=1, br=101.0, pessoas=1)","(ignorados=0, veiculos=1, classificacao_aciden...",0.130542,0.294511,0.130542,1.0,3.395460,0.092096,inf,3,3
4133,"(pessoas=1, br=101.0, classificacao_acidente=C...","(feridos=1, veiculos=1, ignorados=0)",0.130542,0.224842,0.130542,1.0,4.447574,0.101191,inf,3,3
4145,"(br=101.0, feridos=1, pessoas=1)","(mortos=0, ilesos=0, classificacao_acidente=Co...",0.130542,0.346587,0.130542,1.0,2.885279,0.085298,inf,3,3
4146,"(pessoas=1, br=101.0, classificacao_acidente=C...","(feridos=1, mortos=0, ilesos=0)",0.130542,0.225897,0.130542,1.0,4.426791,0.101053,inf,3,3
4160,"(br=101.0, feridos=1, pessoas=1)","(veiculos=1, ilesos=0, classificacao_acidente=...",0.130542,0.260732,0.130542,1.0,3.835358,0.096505,inf,3,3
4161,"(pessoas=1, br=101.0, classificacao_acidente=C...","(veiculos=1, feridos=1, ilesos=0)",0.130542,0.194581,0.130542,1.0,5.139241,0.105141,inf,3,3


In [455]:
# Salvando em um arquivo .csv
saidaRecords.to_csv('AprioriMlxtend_'+ nomeArq + '.csv', encoding= 'ISO-8859-1', sep= ';', index=False)

In [456]:
# saidaRecords.to_excel('XXXAprioriMlxtend3_'+ nomeArq + '.xlsx', index=False)

In [457]:
# Gambiarra para atualizar o arquivo com os dados ordenados, de acordo com a célula anterior
# saidaRecords = pd.read_csv('AprioriMlxtend3_'+ nomeArq + '.csv', encoding='ISO-8859-1', sep=';')

### Filtro para facilitar a visualização dos resultados

In [458]:
# rules[ (rules['antecedent_len'] >= 2) &
#        (rules['confidence'] > 0.75) &
#        (rules['lift'] > 1.2) ]

In [459]:
# rules[rules['antecedents'] == {'Eggs', 'Kidney Beans'}]

## Filtros modelos para serem utilizados no apriori

In [460]:
# saidaRecords[ (saidaRecords['antecedent_len'] >= 3 ) &
#                    (saidaRecords['consequent_len'] >= 3) ]

In [461]:
# frequent_itemsets[ (frequent_itemsets['length'] == 2) &
#                    (frequent_itemsets['support'] >= 0.8) ]

In [462]:
# frequent_itemsets[ frequent_itemsets['itemsets'] == {'Onion', 'Eggs'} ]

# ATIVIDADES FUTURAS

- Verificar se a coluna ['ignorados'] é necessária para analisar os resultados. <br>
Analisar o significado da coluna 'ignorados' no site da PRF. Ou no arquivo de descrição de tabelas.
***Resposta** : No link do anuário 2020, lá é apresentado alguns detalhes dos dados.

- Estudar os parâmetros do algoritmo apriori.

- Analisar e entende melhor a coluna 'pessoas'.

- Analisar a saída do 'lift' e 'conviction'.
***Resposta**: O problema está no momento que os dados 'lift' e 'conviction' são passados para o arquivo .csv.

No Jupyter os dados estão sendo apresentados da maneira correta.

# Print Final

In [463]:
print("TUDO EXECUTADO!")

TUDO EXECUTADO!
